In [47]:
import pandas as pd
import numpy as np
import math
import pickle
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

import random

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
import timeit
from pathlib import Path



import matplotlib.pyplot as plt

import SMOTE
import CFS
import birch


from multiprocessing import Pool, cpu_count
from threading import Thread
from multiprocessing import Queue

import metrics

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [53]:
def cluster_driver(df,print_tree = True):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

def get_clusters(data_source):
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source + '/'
    else:
        _dir = data_source + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    return clusters

def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.iloc[i].name
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-3.5}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n


In [5]:
goals = ['recall','precision','pf']
fold = str(0)

In [29]:
data_source = 'results/median_data/level_2/fold_' + fold
clusters = get_clusters(data_source)
df = pd.read_csv(clusters[0] + '/700_RF_bellwether_' + goals[0] + '.csv', index_col = 0)
    

In [34]:
for row in range(df.shape[0]):
    j = df.iloc[row]
    j.drop(df.index[row], inplace = True)
    print(j.median())

0.84
1.0
0.79
0.89
0.8
1.0
0.99
0.96
0.99
0.96
0.96
1.0


In [32]:
df.index[row]

'embulk'

In [ ]:
KernelAdiutor	0.97	0.9	0.08	14
twilio-java	0.98	0.57	0.305	1
keepassdroid	0.99	0.89	0.075	17

In [39]:
df = pd.read_pickle('results/median_data/level_2/fold_0/cdom_level1/cluster_4_performance.pkl')

In [43]:
for goal in goals:
    new_dict = {}
    for key1 in df.keys():
        if key1 not in new_dict.keys():
            new_dict[key1] = {}
        for key2 in df[key1].keys():
            new_dict[key1][key2] = df[key1][key2][goal][0]
    new_dict_df = pd.DataFrame.from_dict(new_dict, orient = 'index')
    print(goal)
    print(new_dict_df)

recall
               KernelAdiutor  twilio-java  keepassdroid
KernelAdiutor           1.00         0.79          0.99
twilio-java             0.89         1.00          1.00
keepassdroid            0.93         0.95          1.00
precision
               KernelAdiutor  twilio-java  keepassdroid
KernelAdiutor           1.00         0.72          0.92
twilio-java             0.27         0.99          0.87
keepassdroid            0.82         0.59          0.99
pf
               KernelAdiutor  twilio-java  keepassdroid
KernelAdiutor           0.00         0.17          0.06
twilio-java             0.92         0.00          0.11
keepassdroid            0.08         0.36          0.01


In [42]:
new_dict

{'KernelAdiutor': {'KernelAdiutor': 0.0,
  'twilio-java': 0.17,
  'keepassdroid': 0.06},
 'twilio-java': {'KernelAdiutor': 0.92,
  'twilio-java': 0.0,
  'keepassdroid': 0.11},
 'keepassdroid': {'KernelAdiutor': 0.08,
  'twilio-java': 0.36,
  'keepassdroid': 0.01}}

In [61]:
def find_bellwether_level1(data_source,clusters,path,fold):
    df_train = pd.read_pickle(data_source + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['recall','precision','pf']
    score_df = pd.read_csv(data_source + '/bellwether_cdom_2.csv')
    score_df.drop(labels = ['Unnamed: 0'], axis = 1 ,inplace = True)
    level_1_bellwethers = {}
    for cluster in cluster_structure[2].keys():
        if cluster_structure[2][cluster] not in level_1_bellwethers.keys():
            level_1_bellwethers[cluster_structure[2][cluster]] = []
        level_1_bellwethers[cluster_structure[2][cluster]].append(score_df[score_df['id'] == cluster].bellwether.values[0])
    for cluster in cluster_structure[1].keys():
        if cluster not in level_1_bellwethers.keys():
            level_1_bellwethers[cluster] = list(df_train.loc[cluster_tree[cluster].data_points].index)
    bell_df = {}
    for key in  level_1_bellwethers.keys():
        sub_cluster_bellwethers = level_1_bellwethers[key]
        try:
            final_score = pd.read_pickle(data_source + '/cdom_level1/cluster_'  + str(key) + '_performance.pkl')
        except:
            continue
        _results = {}
        for goal in goals:    
            for s_project in final_score.keys():
                if s_project not in _results.keys():
                    _results[s_project] = {}
                _temp = []
                for d_projects in final_score[s_project].keys():
                    if goal == 'g':
                        _goal = 'g-score'
                    else:
                        _goal = goal
                    _score = np.median(final_score[s_project][d_projects][_goal])
                    _temp.append(np.median(final_score[s_project][d_projects][_goal]))
                print(goal,s_project,_temp)
                if goal not in _results[s_project].keys():
                    _results[s_project][goal] = []
                _results[s_project][goal] = np.median(_temp)
        _df = pd.DataFrame.from_dict(_results, orient = 'index')
        print(_df)
        dom_score = []
        for row_id in range(_df.shape[0]):
            project_name = _df.iloc[row_id].name
            row = _df.iloc[row_id].tolist()
            wins = dominate(_df,row,project_name,goals)
            dom_score.append(wins)
        _df['wins'] = dom_score
        c_dom = _df.wins.values.tolist()
        best_project = _df.index[c_dom.index(max(c_dom))]
        best_project_perf = _df.loc[best_project].values.tolist()
        best_project_perf.append(best_project)
        bell_df[key] = best_project_perf
    perf_df = pd.DataFrame.from_dict(bell_df, orient = 'index', columns = ['recall','precision','pf','cdom','bellwether'])

In [62]:
path = '../data/700/merged_data'
data_source = 'results/median_data/level_2/fold_' + fold
conv_bell_perf = 'results/median_data/default_bell/'
conv_bell_loc = 'results/median_data/default_bell/fold_' + fold
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source + '/'
else:
    _dir = data_source + '\\'

clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
find_bellwether_level1(data_source,clusters,path,0)


[cluster_id=0] N_children: 8 N_samples: 711
> [cluster_id=1] N_children: 0 N_samples: 1
> [cluster_id=2] N_children: 0 N_samples: 1
> [cluster_id=3] N_children: 0 N_samples: 9
> [cluster_id=4] N_children: 3 N_samples: 35
> > [cluster_id=5] N_children: 0 N_samples: 15
> > [cluster_id=6] N_children: 0 N_samples: 2
> > [cluster_id=7] N_children: 0 N_samples: 18
> [cluster_id=8] N_children: 4 N_samples: 47
> > [cluster_id=9] N_children: 0 N_samples: 13
> > [cluster_id=10] N_children: 0 N_samples: 9
> > [cluster_id=11] N_children: 0 N_samples: 15
> > [cluster_id=12] N_children: 0 N_samples: 10
> [cluster_id=13] N_children: 18 N_samples: 215
> > [cluster_id=14] N_children: 0 N_samples: 1
> > [cluster_id=15] N_children: 0 N_samples: 9
> > [cluster_id=16] N_children: 0 N_samples: 15
> > [cluster_id=17] N_children: 0 N_samples: 20
> > [cluster_id=18] N_children: 0 N_samples: 14
> > [cluster_id=19] N_children: 0 N_samples: 17
> > [cluster_id=20] N_children: 0 N_samples: 18
> > [cluster_id=21] N_

recall TagMo [1.0, 0.97, 0.96, 0.91, 0.88, 0.71, 0.97, 0.95, 0.92, 0.97, 0.8, 0.91, 0.58]
recall tabula-java [0.99, 1.0, 0.28, 0.86, 0.89, 0.85, 0.94, 0.67, 0.89, 0.81, 0.81, 0.9, 0.15]
recall twoway-view [0.82, 0.81, 1.0, 0.89, 0.91, 0.65, 0.98, 0.86, 0.84, 0.91, 0.81, 0.84, 0.02]
recall sonar-gitlab-plugin [0.96, 0.96, 0.96, 1.0, 0.96, 0.84, 0.88, 0.98, 0.93, 0.9, 0.91, 0.93, 0.58]
recall SimpleCropView [0.92, 0.86, 0.91, 0.83, 1.0, 0.62, 0.88, 0.65, 0.85, 0.93, 0.62, 0.9, 0.62]
recall AndroidSlidingUpPanel [1.0, 0.98, 0.91, 0.97, 0.97, 1.0, 0.98, 0.91, 0.98, 0.92, 1.0, 0.98, 0.88]
recall Pedometer [0.9, 0.93, 1.0, 0.92, 0.98, 0.94, 1.0, 0.95, 0.95, 0.96, 0.78, 0.95, 0.93]
recall ice [0.83, 0.95, 0.99, 0.95, 0.98, 0.68, 0.98, 1.0, 0.92, 0.96, 0.94, 0.97, 0.83]
recall mysql-binlog-connector-java [1.0, 0.97, 0.98, 0.97, 0.99, 0.99, 0.99, 1.0, 1.0, 1.0, 0.92, 0.99, 0.47]
recall snappy-java [0.99, 0.82, 0.96, 0.81, 0.98, 0.88, 0.93, 1.0, 0.89, 1.0, 0.83, 0.93, 0.75]
recall PokeGOAPI-Java

precision algorithms-sedgewick-wayne [0.14, 0.97, 0.94, 0.63, 0.85, 0.97, 0.96, 0.81, 0.85, 0.87, 0.95, 0.61, 0.86, 0.57, 0.94, 0.91, 0.92, 0.95, 0.83, 0.94]
precision togglz [0.64, 0.98, 0.81, 0.87, 0.87, 0.97, 0.92, 1.0, 0.84, 0.78, 0.55, 0.33, 0.83, 0.96, 0.87, 0.89, 0.88, 0.95, 0.87, 0.94]
precision tilesfx [0.23, 0.79, 0.7, 0.76, 0.55, 0.63, 0.64, 0.61, 1.0, 0.36, 0.44, 0.8, 0.35, 0.95, 0.32, 0.61, 0.74, 0.84, 0.79, 0.63]
precision android-smsmms [0.42, 0.94, 0.87, 0.85, 0.91, 0.98, 0.9, 0.82, 0.85, 0.99, 0.9, 0.88, 0.86, 0.95, 0.92, 0.89, 0.86, 0.94, 0.85, 0.93]
precision spring-cloud-aws [0.2, 0.92, 0.69, 0.81, 0.88, 0.96, 0.65, 0.75, 0.83, 0.73, 0.98, 0.58, 0.77, 0.92, 0.85, 0.89, 0.86, 0.88, 0.84, 0.93]
precision java-driver [0.47, 0.91, 0.72, 0.81, 0.87, 0.96, 0.78, 0.81, 0.84, 0.79, 0.78, 1.0, 0.84, 0.95, 0.89, 0.89, 0.87, 0.95, 0.82, 0.88]
precision bootique [0.28, 0.96, 0.64, 0.82, 0.88, 0.96, 0.83, 0.84, 0.84, 0.78, 0.89, 0.85, 1.0, 0.93, 0.84, 0.9, 0.87, 0.94, 0.86, 0.59